In [1]:
pip install scikit-learn xgboost lightgbm catboost joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib
import optuna

c:\Users\NongNam\Documents\AI_Builder\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load the dataset you created
df = pd.read_csv("descriptor.csv")
df = df.copy()
df = df[df['Kd'].notnull()]  # Remove NaN
df = df[np.isfinite(df['Kd'])]  # Remove inf/-inf


In [4]:
df.keys()

Index(['SlogP_VSA5', 'BCUT2D_CHGHI', 'SMR_VSA5', 'MinEStateIndex', 'Chi4v',
       'Prot_MW', 'Aromaticity', 'Instability', 'Hydropathy',
       'IsoelectricPoint', 'Kd'],
      dtype='object')

In [5]:
features = ['SlogP_VSA5', 'BCUT2D_CHGHI', 'SMR_VSA5', 'MinEStateIndex', 'Chi4v','Prot_MW', 'Aromaticity', 'Instability', 'Hydropathy','IsoelectricPoint']
target = 'Kd'

X = df[features]
y = df[target]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.22, random_state=42)


In [6]:
# Objective functions
def rf_objective(trial):
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RandomForestRegressor(
            n_estimators=trial.suggest_categorical('n_estimators', [100, 200]),
            max_depth=trial.suggest_categorical('max_depth', [5, 10]),
            random_state=42
        ))
    ])
    return cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean()

def xgb_objective(trial):
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', XGBRegressor(
            n_estimators=trial.suggest_categorical('n_estimators', [100, 200]),
            max_depth=trial.suggest_categorical('max_depth', [4, 6]),
            learning_rate=trial.suggest_categorical('learning_rate', [0.05, 0.1]),
            random_state=42, verbosity=0
        ))
    ])
    return cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean()

def lgb_objective(trial):
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', LGBMRegressor(
            n_estimators=trial.suggest_categorical('n_estimators', [100, 200]),
            max_depth=trial.suggest_categorical('max_depth', [4, 6]),
            learning_rate=trial.suggest_categorical('learning_rate', [0.05, 0.1]),
            random_state=42
        ))
    ])
    return cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean()

def cat_objective(trial):
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('model', CatBoostRegressor(
            iterations=trial.suggest_categorical('iterations', [100, 200]),
            depth=trial.suggest_categorical('depth', [4, 6]),
            learning_rate=trial.suggest_categorical('learning_rate', [0.05, 0.1]),
            verbose=0,
            random_state=42
        ))
    ])
    return cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_squared_error').mean()


In [7]:
# Train and save best models
objectives = {'rf': rf_objective, 'xgb': xgb_objective, 'lgb': lgb_objective, 'cat': cat_objective}
oof_preds = []
model_paths = {}

In [8]:
for name, obj_func in objectives.items():
    print(f"Running Optuna tuning for {name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(obj_func, n_trials=10)

    print(f"Best params for {name}: {study.best_params}")

    if name == 'rf':
        model = RandomForestRegressor(**study.best_params, random_state=42)
    elif name == 'xgb':
        model = XGBRegressor(**study.best_params, random_state=42, verbosity=0)
    elif name == 'lgb':
        model = LGBMRegressor(**study.best_params, random_state=42)
    elif name == 'cat':
        model = CatBoostRegressor(**study.best_params, verbose=0, random_state=42)

    pipeline = Pipeline([('scaler', StandardScaler()), ('model', model)])
    pipeline.fit(X_train, y_train)
    preds = pipeline.predict(X_val)
    mse = mean_squared_error(y_val, preds)
    print(f"{name.upper()} MSE: {mse:.4f}")

    joblib.dump(pipeline, f"{name}_optuna_model.pkl")
    model_paths[name] = f"{name}_optuna_model.pkl"
    oof_preds.append(preds.reshape(-1, 1))


[I 2025-06-02 15:00:26,118] A new study created in memory with name: no-name-0e1b5b47-f22c-4013-bd6d-6d9ca4d99927


Running Optuna tuning for RF...


[I 2025-06-02 15:01:01,093] Trial 0 finished with value: -1.9086487699046304 and parameters: {'n_estimators': 200, 'max_depth': 5}. Best is trial 0 with value: -1.9086487699046304.
[I 2025-06-02 15:02:07,541] Trial 1 finished with value: -1.5811556942033205 and parameters: {'n_estimators': 200, 'max_depth': 10}. Best is trial 1 with value: -1.5811556942033205.
[I 2025-06-02 15:02:53,983] Trial 2 finished with value: -1.9086487699046304 and parameters: {'n_estimators': 200, 'max_depth': 5}. Best is trial 1 with value: -1.5811556942033205.
[I 2025-06-02 15:03:24,508] Trial 3 finished with value: -1.5806399063875343 and parameters: {'n_estimators': 100, 'max_depth': 10}. Best is trial 3 with value: -1.5806399063875343.
[I 2025-06-02 15:04:29,568] Trial 4 finished with value: -1.5811556942033205 and parameters: {'n_estimators': 200, 'max_depth': 10}. Best is trial 3 with value: -1.5806399063875343.
[I 2025-06-02 15:05:29,745] Trial 5 finished with value: -1.5811556942033205 and parameters:

Best params for rf: {'n_estimators': 100, 'max_depth': 10}


[I 2025-06-02 15:08:14,533] A new study created in memory with name: no-name-0d7ff106-4677-4ff8-9108-3a33b32f2725


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000799 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2549
[LightGBM] [Info] Number of data points in the train set: 28529, number of used features: 10
[LightGBM] [Info] Start training from score 6.051104
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

c:\Users\NongNam\Documents\AI_Builder\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-06-02 15:08:25,760] A new study created in memory with name: no-name-5212b447-865b-4569-a88c-906982143fd4


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-06-02 15:08:28,919] Trial 0 finished with value: -1.6928998348913955 and parameters: {'iterations': 200, 'depth': 6, 'learning_rate': 0.05}. Best is trial 0 with value: -1.6928998348913955.
[I 2025-06-02 15:08:30,893] Trial 1 finished with value: -1.697847939382976 and parameters: {'iterations': 200, 'depth': 4, 'learning_rate': 0.1}. Best is trial 0 with value: -1.6928998348913955.
[I 2025-06-02 15:08:32,426] Trial 2 finished with value: -1.802314524798379 and parameters: {'iterations': 100, 'depth': 6, 'learning_rate': 0.05}. Best is trial 0 with value: -1.6928998348913955.
[I 2025-06-02 15:08:34,393] Trial 3 finished with value: -1.697847939382976 and parameters: {'iterations': 200, 'depth': 4, 'learning_rate': 0.1}. Best is trial 0 with value: -1.6928998348913955.
[I 2025-06-02 15:08:37,364] Trial 4 finished with value: -1.6928998348913955 and parameters: {'iterations': 200, 'depth': 6, 'learning_rate': 0.05}. Best is trial 0 with value: -1.6928998348913955.
[I 2025-06-02 1

Best params for cat: {'iterations': 200, 'depth': 6, 'learning_rate': 0.1}
CAT MSE: 1.4862


In [12]:
# Train meta-model (stacking)
X_meta = np.hstack(oof_preds)
meta_model = LinearRegression()
meta_model.fit(X_meta, y_val)
meta_preds = meta_model.predict(X_meta)
meta_mse = mean_squared_error(y_val, meta_preds)
print(f"Meta-model (stacking) MSE: {meta_mse:.4f}")


Meta-model (stacking) MSE: 1.4165


In [13]:
joblib.dump(meta_model, "meta_model_optuna.pkl")
np.save("stacking_features_optuna.npy", X_meta)